# Country Performance Analysis

This notebook analyzes forecasting model performance at the country level by:
1. Loading predictions from 2022-2024
2. Merging with country information
3. Calculating metrics per country
4. Identifying poorly performing countries (accuracy <= 0.35) for separate modeling

## Cell 1: Imports and Setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from food_crisis_functions import all_metrics
import warnings
warnings.filterwarnings('ignore')

print("Imports successful!")

Imports successful!


## Cell 2: Load Three Prediction CSV Files

In [3]:
# Define file paths (using WSL-compatible paths)
base_path = r"C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\2.source_code\Step5_Geo_RF_trial\IPCCH_model"

pred_2022 = f"{base_path}/forecasting_prediction/forecasting_y_pred_test_2022.csv"
pred_2023 = f"{base_path}/forecasting_prediction/forecasting_y_pred_test_2023.csv"
pred_2024 = f"{base_path}/forecasting_prediction/forecasting_y_pred_test_2024.csv"

# Load all three files
print("Loading prediction files...")
df_2022 = pd.read_csv(pred_2022)
print(f"2022: {len(df_2022):,} rows, {len(df_2022.columns)} columns")

df_2023 = pd.read_csv(pred_2023)
print(f"2023: {len(df_2023):,} rows, {len(df_2023.columns)} columns")

df_2024 = pd.read_csv(pred_2024)
print(f"2024: {len(df_2024):,} rows, {len(df_2024.columns)} columns")

# Display column names
print("\n2022 columns:", df_2022.columns.tolist())
print("2023 columns:", df_2023.columns.tolist())
print("2024 columns:", df_2024.columns.tolist())

Loading prediction files...
2022: 747,720 rows, 14 columns
2023: 886,500 rows, 15 columns
2024: 773,100 rows, 15 columns

2022 columns: ['phase2_pred', 'phase2_test', 'phase3_pred', 'phase3_test', 'phase4_pred', 'phase4_test', 'phase5_test', 'test_index', 'overall_phase', 'overall_phase_pred', 'area_id', 'date', 'lat', 'lon']
2023 columns: ['phase2_pred', 'phase2_test', 'phase3_pred', 'phase3_test', 'phase4_pred', 'phase4_test', 'phase5_pred', 'phase5_test', 'test_index', 'overall_phase', 'overall_phase_pred', 'area_id', 'date', 'lat', 'lon']
2024 columns: ['phase2_pred', 'phase2_test', 'phase3_pred', 'phase3_test', 'phase4_pred', 'phase4_test', 'phase5_pred', 'phase5_test', 'test_index', 'overall_phase', 'overall_phase_pred', 'area_id', 'date', 'lat', 'lon']


## Cell 3: Add Missing phase5_pred to 2022 Data

In [4]:
# Check if phase5_pred is missing in 2022
if 'phase5_pred' not in df_2022.columns:
    df_2022['phase5_pred'] = 0.0
    print("Added phase5_pred column to 2022 data with value 0.0")
else:
    print("phase5_pred already exists in 2022 data")

# Verify all dataframes now have the same structure for phase predictions
required_cols = ['phase2_pred', 'phase2_test', 'phase3_pred', 'phase3_test', 
                 'phase4_pred', 'phase4_test', 'phase5_pred', 'phase5_test',
                 'overall_phase', 'overall_phase_pred', 'area_id', 'date']

for year, df in [(2022, df_2022), (2023, df_2023), (2024, df_2024)]:
    missing = [col for col in required_cols if col not in df.columns]
    if missing:
        print(f"WARNING: {year} missing columns: {missing}")
    else:
        print(f"{year}: All required columns present")

Added phase5_pred column to 2022 data with value 0.0
2022: All required columns present
2023: All required columns present
2024: All required columns present


## Cell 4: Drop Duplicates and Combine Years

In [5]:
# Check duplicates before dropping
print("Before deduplication:")
print(f"2022: {len(df_2022):,} rows")
print(f"2023: {len(df_2023):,} rows")
print(f"2024: {len(df_2024):,} rows")
print(f"Total: {len(df_2022) + len(df_2023) + len(df_2024):,} rows")

# Drop duplicates based on key columns
dedup_cols = ['area_id', 'date', 'overall_phase', 'overall_phase_pred']

df_2022 = df_2022.drop_duplicates(subset=dedup_cols, keep='first').reset_index(drop=True)
df_2023 = df_2023.drop_duplicates(subset=dedup_cols, keep='first').reset_index(drop=True)
df_2024 = df_2024.drop_duplicates(subset=dedup_cols, keep='first').reset_index(drop=True)

print("\nAfter deduplication:")
print(f"2022: {len(df_2022):,} rows")
print(f"2023: {len(df_2023):,} rows")
print(f"2024: {len(df_2024):,} rows")

# Add year column to track source
df_2022['year'] = 2022
df_2023['year'] = 2023
df_2024['year'] = 2024

# Concatenate all dataframes
df_combined = pd.concat([df_2022, df_2023, df_2024], axis=0, ignore_index=True)

print(f"\nCombined dataframe: {len(df_combined):,} rows")
print(f"Unique area_ids: {df_combined['area_id'].nunique()}")
print(f"Date range: {df_combined['date'].min()} to {df_combined['date'].max()}")

# Display first few rows
df_combined.head()

Before deduplication:
2022: 747,720 rows
2023: 886,500 rows
2024: 773,100 rows
Total: 2,407,320 rows

After deduplication:
2022: 4,154 rows
2023: 4,925 rows
2024: 4,295 rows

Combined dataframe: 13,374 rows
Unique area_ids: 4237
Date range: 2022-01-01 to 2024-11-01


,phase2_pred,phase2_test,phase3_pred,phase3_test,phase4_pred,phase4_test,phase5_test,test_index,overall_phase,overall_phase_pred,area_id,date,lat,lon,phase5_pred,year
0,0.61,0.60,0.25,0.25,0.04,0.0,0.0,19032.0,3.0,3.0,11,2022-07-01,-30.360068,27.985838,0.0,2022
1,0.55,0.55,0.25,0.25,0.04,0.0,0.0,19023.0,3.0,3.0,12,2022-07-01,-30.069920,27.800794,0.0,2022
2,0.64,0.65,0.24,0.25,0.04,0.0,0.0,18803.0,3.0,3.0,14,2022-07-01,-29.966109,28.737390,0.0,2022
3,0.69,0.60,0.28,0.20,0.04,0.0,0.0,18793.0,3.0,3.0,16,2022-07-01,-29.789284,27.442293,0.0,2022
4,0.67,0.60,0.25,0.25,0.04,0.0,0.0,18784.0,3.0,3.0,17,2022-07-01,-29.581692,27.824245,0.0,2022


## Cell 5: Load Source Data and Merge for Country Information

In [7]:
# Load source data
source_path = r"C:\Users\swl00\IFPRI Dropbox\Weilun Shi\Google fund\Analysis\1.Source Data\IPCCH_2017_2025_final_v12102025_with_zscores.csv"

print("Loading source data...")
# Only load the columns we need to save memory
df_source = pd.read_csv(source_path, usecols=['admin_code', 'country', 'country_code', 'country_en'])
print(f"Source data loaded: {len(df_source):,} rows")

# Extract unique area_id to country mapping
area_country_map = df_source.drop_duplicates(subset='admin_code').copy()
print(f"Unique admin_codes: {len(area_country_map):,}")

# Rename admin_code to area_id for merging
area_country_map = area_country_map.rename(columns={'admin_code': 'area_id'})

# Ensure area_id types match
df_combined['area_id'] = df_combined['area_id'].astype(int)
area_country_map['area_id'] = area_country_map['area_id'].astype(int)

# Left join to preserve all forecasting rows
print("\nMerging forecasting data with country information...")
df_merged = df_combined.merge(
    area_country_map,
    on='area_id',
    how='left',
    validate='m:1'  # Many predictions to one area_id
)

print(f"Merged dataframe: {len(df_merged):,} rows")

# Check for unmatched area_ids
unmatched = df_merged['country'].isna().sum()
if unmatched > 0:
    print(f"\nWARNING: {unmatched} rows have no country match!")
    print(f"Unmatched area_ids: {sorted(df_merged[df_merged['country'].isna()]['area_id'].unique())}")
else:
    print(f"\nSuccess: All {len(df_merged):,} rows matched to countries")

print(f"Total unique countries: {df_merged['country'].nunique()}")
print(f"\nCountries in dataset:")
print(df_merged['country'].value_counts())

Loading source data...
Source data loaded: 1,120,860 rows
Unique admin_codes: 6,227

Merging forecasting data with country information...
Merged dataframe: 13,374 rows

Success: All 13,374 rows matched to countries
Total unique countries: 49

Countries in dataset:
country
Nigeria                             3012
Somalia                             2448
Central African Republic             642
Democratic Republic of the Congo     579
Sudan                                528
Chad                                 397
Benin                                348
Cameroon                             329
South Sudan                          311
Niger                                305
Mozambique                           295
Ghana                                293
Senegal                              261
Zambia                               254
Afghanistan                          253
Mali                                 244
Yemen                                237
Lebanon                       

## Cell 6: Validation Checks

In [8]:
print("=" * 80)
print("DATA VALIDATION CHECKS")
print("=" * 80)

# Check 1: Merge success rate
merge_success = (1 - df_merged['country'].isna().sum() / len(df_merged)) * 100
print(f"\n1. Merge success rate: {merge_success:.2f}%")

# Check 2: Overall phase distribution
print("\n2. Overall phase distribution (actual):")
print(df_merged['overall_phase'].value_counts().sort_index())

print("\n3. Overall phase prediction distribution:")
print(df_merged['overall_phase_pred'].value_counts().sort_index())

# Check 3: Check for NaN in critical columns
critical_cols = ['phase3_test', 'phase3_pred', 'overall_phase', 'overall_phase_pred']
print("\n4. NaN values in critical columns:")
for col in critical_cols:
    nan_count = df_merged[col].isna().sum()
    print(f"   {col}: {nan_count} NaN values")
    if nan_count > 0:
        print(f"      WARNING: {col} has {nan_count} NaN values!")

# Check 4: Year distribution
print("\n5. Predictions by year:")
print(df_merged['year'].value_counts().sort_index())

# Check 5: Data quality summary
print("\n6. Data quality summary:")
print(f"   Total predictions: {len(df_merged):,}")
print(f"   Unique area_ids: {df_merged['area_id'].nunique()}")
print(f"   Unique countries: {df_merged['country'].nunique()}")
print(f"   Date range: {df_merged['date'].min()} to {df_merged['date'].max()}")

print("\n" + "=" * 80)

DATA VALIDATION CHECKS

1. Merge success rate: 100.00%

2. Overall phase distribution (actual):
overall_phase
1.0    2179
2.0    5612
3.0    4683
4.0     874
5.0      26
Name: count, dtype: int64

3. Overall phase prediction distribution:
overall_phase_pred
1.0     575
2.0    6781
3.0    5837
4.0     181
Name: count, dtype: int64

4. NaN values in critical columns:
   phase3_test: 0 NaN values
   phase3_pred: 0 NaN values
   overall_phase: 0 NaN values
   overall_phase_pred: 0 NaN values

5. Predictions by year:
year
2022    4154
2023    4925
2024    4295
Name: count, dtype: int64

6. Data quality summary:
   Total predictions: 13,374
   Unique area_ids: 4237
   Unique countries: 49
   Date range: 2022-01-01 to 2024-11-01



## Cell 7: Calculate all_metrics Per Country

In [9]:
# Initialize results list
results = []

# Get unique countries (exclude NaN)
countries = df_merged['country'].dropna().unique()
print(f"Calculating metrics for {len(countries)} countries...\n")

# Process each country
for i, country in enumerate(countries, 1):
    # Filter data for this country (all years combined)
    country_data = df_merged[df_merged['country'] == country].copy()
    
    # Skip if insufficient data
    if len(country_data) < 5:
        print(f"[{i}/{len(countries)}] Skipping {country}: only {len(country_data)} predictions")
        continue
    
    # Extract arrays for all_metrics function
    y_test = country_data['overall_phase'].values
    y_pred = country_data['overall_phase_pred'].values
    
    # Create confusion matrix with labels 1-5
    cm = confusion_matrix(y_test, y_pred, labels=[1, 2, 3, 4, 5])
    
    # Create y_pred_test dataframe with required columns
    y_pred_test_df = country_data[['phase3_test', 'phase3_pred']].copy()
    
    # Calculate metrics
    try:
        accuracy, sensitivity, precision, r2 = all_metrics(y_test, y_pred, cm, y_pred_test_df)
        
        # Store results
        results.append({
            'country': country,
            'country_code': country_data['country_code'].iloc[0] if 'country_code' in country_data.columns else None,
            'n_predictions': len(country_data),
            'n_areas': country_data['area_id'].nunique(),
            'accuracy': accuracy,
            'sensitivity': sensitivity,
            'precision': precision,
            'r2': r2,
            'years_covered': sorted(country_data['year'].unique().tolist())
        })
        
        # Print progress every 10 countries
        if i % 10 == 0:
            print(f"[{i}/{len(countries)}] Processed {country}: accuracy={accuracy:.3f}")
            
    except Exception as e:
        print(f"[{i}/{len(countries)}] Error calculating metrics for {country}: {e}")
        continue

# Create results dataframe
results_df = pd.DataFrame(results)

print(f"\n{'='*80}")
print(f"Metrics calculated for {len(results_df)} countries")
print(f"{'='*80}")

# Sort by accuracy
results_df = results_df.sort_values('accuracy')

# Display summary statistics
print("\nAccuracy statistics:")
print(results_df['accuracy'].describe())

# Display first 10 rows
print("\nFirst 10 countries (worst performers):")
results_df.head(10)

Calculating metrics for 49 countries...

[10/49] Processed Tanzania: accuracy=0.940
[20/49] Processed Guatemala: accuracy=0.643
[30/49] Processed Liberia: accuracy=0.733
[40/49] Processed Guinea-Bissau: accuracy=0.422
[46/49] Skipping El Salvador: only 2 predictions

Metrics calculated for 48 countries

Accuracy statistics:
count    48.000000
mean      0.611739
std       0.157723
min       0.083333
25%       0.533619
50%       0.599905
75%       0.707744
max       0.940000
Name: accuracy, dtype: float64

First 10 countries (worst performers):


,country,country_code,n_predictions,n_areas,accuracy,sensitivity,precision,r2,years_covered
46,Palestinian Territory,PS,12,5,0.083333,1.000000,1.000000,-164.108000,"[2023, 2024]"
38,Mali,ML,244,49,0.340164,0.450000,0.333333,-0.274110,"[2022, 2023, 2024]"
12,Somalia,SO,2448,804,0.402369,0.739187,0.612627,-0.236872,"[2022, 2023, 2024]"
36,Ghana,GH,293,66,0.419795,0.000000,0.000000,0.009687,"[2022, 2023, 2024]"
39,Guinea-Bissau,GW,45,8,0.422222,0.000000,0.000000,-2.186028,"[2022, 2023, 2024]"
43,Cabo Verde,CV,82,22,0.451220,0.000000,0.000000,-0.204209,"[2022, 2023, 2024]"
11,Kenya,KE,145,35,0.468966,0.729167,0.411765,0.009564,"[2022, 2023, 2024]"
30,Cote d'Ivoire,CI,163,31,0.472393,0.000000,0.000000,-0.446116,"[2022, 2023, 2024]"
6,Malawi,MW,96,32,0.500000,0.409091,0.750000,-0.028873,"[2022, 2023, 2024]"
13,Uganda,UG,81,42,0.518519,0.863636,0.535211,-0.436987,"[2022, 2023, 2024]"


## Cell 8: Filter and Display Poor Performers (Accuracy <= 0.35)

In [10]:
# Filter for poorly performing countries
poor_performance = results_df[results_df['accuracy'] <= 0.35].copy()

# Sort by accuracy (worst first)
poor_performance = poor_performance.sort_values('accuracy')

print("=" * 80)
print("COUNTRIES WITH ACCURACY <= 0.35")
print("=" * 80)
print(f"\nTotal: {len(poor_performance)} countries out of {len(results_df)} analyzed ({len(poor_performance)/len(results_df)*100:.1f}%)\n")

if len(poor_performance) > 0:
    # Display full table
    print(poor_performance.to_string(index=False))
    
    print(f"\n{'='*80}")
    print("SUMMARY FOR POOR PERFORMERS")
    print(f"{'='*80}")
    print(f"\nAccuracy range: {poor_performance['accuracy'].min():.3f} - {poor_performance['accuracy'].max():.3f}")
    print(f"Mean accuracy: {poor_performance['accuracy'].mean():.3f}")
    print(f"Total predictions: {poor_performance['n_predictions'].sum():,}")
    print(f"Total areas: {poor_performance['n_areas'].sum()}")
    
else:
    print("No countries found with accuracy <= 0.35")
    print("All countries are performing above the threshold!")

COUNTRIES WITH ACCURACY <= 0.35

Total: 2 countries out of 48 analyzed (4.2%)

              country country_code  n_predictions  n_areas  accuracy  sensitivity  precision         r2      years_covered
Palestinian Territory           PS             12        5  0.083333         1.00   1.000000 -164.10800       [2023, 2024]
                 Mali           ML            244       49  0.340164         0.45   0.333333   -0.27411 [2022, 2023, 2024]

SUMMARY FOR POOR PERFORMERS

Accuracy range: 0.083 - 0.340
Mean accuracy: 0.212
Total predictions: 256
Total areas: 54


## Cell 9: Save Output Files

In [11]:
# 1. Save full results CSV (all countries)
output_path_all = 'country_performance_all.csv'
results_df.to_csv(output_path_all, index=False)
print(f"Saved full results to: {output_path_all}")
print(f"  - {len(results_df)} countries")

# 2. Save poor performance CSV (accuracy <= 0.35)
output_path_poor = 'country_performance_poor.csv'
poor_performance.to_csv(output_path_poor, index=False)
print(f"\nSaved poor performance results to: {output_path_poor}")
print(f"  - {len(poor_performance)} countries")

# 3. Save text file with just country names (for easy reference)
if len(poor_performance) > 0:
    output_path_list = 'poor_performing_countries.txt'
    with open(output_path_list, 'w') as f:
        f.write(f"Countries with Accuracy <= 0.35 ({len(poor_performance)} total)\n")
        f.write("=" * 80 + "\n\n")
        f.write(f"{'Country':<40} {'Accuracy':>10} {'N Predictions':>15} {'N Areas':>10}\n")
        f.write("-" * 80 + "\n")
        for idx, row in poor_performance.iterrows():
            f.write(f"{row['country']:<40} {row['accuracy']:>10.3f} {row['n_predictions']:>15,} {row['n_areas']:>10}\n")
    print(f"\nSaved country list to: {output_path_list}")

# 4. Display final summary
print("\n" + "=" * 80)
print("FINAL SUMMARY STATISTICS")
print("=" * 80)
print(f"\nTotal countries analyzed: {len(results_df)}")
print(f"Countries with accuracy <= 0.35: {len(poor_performance)} ({len(poor_performance)/len(results_df)*100:.1f}%)")
print(f"Countries with accuracy > 0.35: {len(results_df) - len(poor_performance)} ({(len(results_df) - len(poor_performance))/len(results_df)*100:.1f}%)")

print("\nAccuracy distribution (all countries):")
print(f"  Mean:   {results_df['accuracy'].mean():.3f}")
print(f"  Median: {results_df['accuracy'].median():.3f}")
print(f"  Std:    {results_df['accuracy'].std():.3f}")
print(f"  Min:    {results_df['accuracy'].min():.3f}")
print(f"  Max:    {results_df['accuracy'].max():.3f}")

print("\nOther metrics (all countries):")
print(f"  Sensitivity: {results_df['sensitivity'].mean():.3f} ± {results_df['sensitivity'].std():.3f}")
print(f"  Precision:   {results_df['precision'].mean():.3f} ± {results_df['precision'].std():.3f}")
print(f"  R²:          {results_df['r2'].mean():.3f} ± {results_df['r2'].std():.3f}")

print("\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)
print("\nThese countries will need separate models for improved performance.")

Saved full results to: country_performance_all.csv
  - 48 countries

Saved poor performance results to: country_performance_poor.csv
  - 2 countries

Saved country list to: poor_performing_countries.txt

FINAL SUMMARY STATISTICS

Total countries analyzed: 48
Countries with accuracy <= 0.35: 2 (4.2%)
Countries with accuracy > 0.35: 46 (95.8%)

Accuracy distribution (all countries):
  Mean:   0.612
  Median: 0.600
  Std:    0.158
  Min:    0.083
  Max:    0.940

Other metrics (all countries):
  Sensitivity: 0.544 ± 0.409
  Precision:   0.497 ± 0.385
  R²:          -3.834 ± 23.658

ANALYSIS COMPLETE

These countries will need separate models for improved performance.
